In [1]:
import torch
import torch.nn as nn
from attention import AbsolutePositionalEncoding as PositionalEncoding

class TransformerConfig:
    def __init__(self, d_model=256, num_heads=4, num_layers=6, vocab_size=10000, max_len=5000, dropout=0.1):
        self.d_model = d_model
        self.num_heads = num_heads
        self.num_layers = num_layers
        self.vocab_size = vocab_size
        self.max_len = max_len
        self.dropout = dropout

class TransformerEncoder(nn.Module):
    """
    A simple Transformer encoder block.
    """
    def __init__(self, config):
        super().__init__()
        self.embed = nn.Embedding(config.vocab_size, config.d_model)
        self.pos_encoding = PositionalEncoding(config)

class Transformer(nn.Module):
    """
    A simple Transformer model, as in Vaswani et al. (2017).
    """
    def __init__(self, config):
        super().__init__()
        self.encoder = TransformerEncoder(config)

In [11]:
'''
Absolute Positional Encoding
'''

import torch
import math

max_len = 100
d_model = 12
pe = torch.zeros(max_len, d_model)
position = torch.arange(0, max_len, dtype=float).unsqueeze(1)
print("position.shape", position.shape, position)
every_other_dim = torch.arange(0, d_model, 2, dtype=float)
print("every_other_dim.shape", every_other_dim.shape, every_other_dim)
'''
a = 2i/d_model
-ln(10000^a) = -a ln(10000) 
exp(ln(10000^-a)) = 10000^(-a) 
'''
div_term = torch.exp((math.log(10000.0)) * -every_other_dim / d_model)
pe[:, 0::2] = torch.sin(position * div_term)
pe[:, 1::2] = torch.cos(position * div_term)
pe = pe.unsqueeze(0)
pe.shape, pe[0][1]

# pe[0]



position.shape torch.Size([100, 1]) tensor([[ 0.],
        [ 1.],
        [ 2.],
        [ 3.],
        [ 4.],
        [ 5.],
        [ 6.],
        [ 7.],
        [ 8.],
        [ 9.],
        [10.],
        [11.],
        [12.],
        [13.],
        [14.],
        [15.],
        [16.],
        [17.],
        [18.],
        [19.],
        [20.],
        [21.],
        [22.],
        [23.],
        [24.],
        [25.],
        [26.],
        [27.],
        [28.],
        [29.],
        [30.],
        [31.],
        [32.],
        [33.],
        [34.],
        [35.],
        [36.],
        [37.],
        [38.],
        [39.],
        [40.],
        [41.],
        [42.],
        [43.],
        [44.],
        [45.],
        [46.],
        [47.],
        [48.],
        [49.],
        [50.],
        [51.],
        [52.],
        [53.],
        [54.],
        [55.],
        [56.],
        [57.],
        [58.],
        [59.],
        [60.],
        [61.],
        [62.],
        [63.],
    

(torch.Size([1, 100, 12]),
 tensor([8.4147e-01, 5.4030e-01, 2.1378e-01, 9.7688e-01, 4.6399e-02, 9.9892e-01,
         9.9998e-03, 9.9995e-01, 2.1544e-03, 1.0000e+00, 4.6416e-04, 1.0000e+00]))

In [16]:
import torch
import torch.nn as nn
import math


class RelativeMultiHeadSelfAttention(nn.Module):
    """
    An example implementation of multi-head self-attention with
    relative position representations, following Shaw et al. (2018).
    """
    def __init__(self, d_model=256, num_heads=4, max_len=512):
        super().__init__()

        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.max_len = max_len

        # Projections for the usual Q, K, V
        self.query_proj = nn.Linear(d_model, d_model)
        self.key_proj   = nn.Linear(d_model, d_model)
        self.value_proj = nn.Linear(d_model, d_model)
        self.out_proj   = nn.Linear(d_model, d_model)

        self.relative_key_embeddings = nn.Embedding(2 * max_len + 1, self.head_dim)
        self.relative_value_embeddings = nn.Embedding(2 * max_len + 1, self.head_dim)
    
    def forward(self, x):
        batch_size, seq_len, d_model = x.shape

        q = self.query_proj(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.key_proj(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        v = self.value_proj(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)

        # --- 1. Content-based (standard) attention score ---
        # scores shape: (batch_size, num_heads, seq_len, seq_len)
        scores = torch.matmul(q, k.transpose(-2, -1))
        # --- 2. Incorporate relative position (Key) ---
        # Build a matrix of relative position offsets for each pair (i, j)
        pos_ids = torch.arange(seq_len).unsqueeze(1) - torch.arange(seq_len)

        # If sequence is seq_len, then pos_ids is in range [-(seq_len-1), (seq_len-1)]
        # Shifting by max_len - 1 puts the range in [0, 2*(max_len-1)]
        # which is the range of the relative key embeddings
        # that covers all relative distances [-(max_len-1), max_len-1]
        # Further, clamping to 2*(max_len-1) ensures that we do not go out of bounds
        pos_ids = pos_ids + (self.max_len - 1)
        pos_ids = pos_ids.clamp(0, 2*self.max_len + 1) # shape (seq_len, seq_len)

        rel_k = self.relative_key_embeddings(pos_ids) # shape (seq_len, seq_len, head_dim)
        scores_r = torch.matmul(q, rel_k.transpose(-2, -1))
        
        

In [17]:
batch_size = 2
seq_len = 12
d_model = 16
num_heads = 4

# Dummy input
x = torch.randn(batch_size, seq_len, d_model)  # (B, L, d_model)

# Create the module
attn = RelativeMultiHeadSelfAttention(d_model, num_heads, max_len=seq_len)

# Forward pass
output = attn(x)
output

pos_ids.shape torch.Size([12, 12]) tensor([[11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0],
        [12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1],
        [13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2],
        [14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3],
        [15, 14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4],
        [16, 15, 14, 13, 12, 11, 10,  9,  8,  7,  6,  5],
        [17, 16, 15, 14, 13, 12, 11, 10,  9,  8,  7,  6],
        [18, 17, 16, 15, 14, 13, 12, 11, 10,  9,  8,  7],
        [19, 18, 17, 16, 15, 14, 13, 12, 11, 10,  9,  8],
        [20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10,  9],
        [21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10],
        [22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11]])
